In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sqlite3
from src.db_manager import SQLiteWrapper
from difflib import SequenceMatcher

pd.options.display.max_columns = None
pd.options.display.max_rows = None

## 1. Cargar ambas BDs

In [2]:
# Cargar ambas bases de datos
db_nuevo = SQLiteWrapper("data/KoboReader_nuevo.sqlite")
db_nuevo.connect()

db_antiguo = SQLiteWrapper("data/KoboReader.sqlite")
db_antiguo.connect()

print('✓ BD nueva conectada')
print('✓ BD antigua conectada')

✓ BD nueva conectada
✓ BD antigua conectada


## 2. Obtener libros con anotaciones en BD antigua

In [3]:
# Obtener libros que TIENEN anotaciones en BD antigua
libros_con_anotaciones_antiguo = db_antiguo.get_query_df("""
    SELECT DISTINCT 
        c.BookID,
        c.BookTitle,
        COUNT(b.BookmarkID) as num_anotaciones
    FROM content c
    LEFT JOIN Bookmark b ON c.BookID = b.VolumeID
    WHERE b.BookmarkID IS NOT NULL
    GROUP BY c.BookID, c.BookTitle
    ORDER BY num_anotaciones DESC
""")

print(f'Libros con anotaciones en BD antigua: {len(libros_con_anotaciones_antiguo)}')
print(f'Total de anotaciones: {libros_con_anotaciones_antiguo["num_anotaciones"].sum()}')
print('\nTop 10:')
display(libros_con_anotaciones_antiguo.head(10))

Libros con anotaciones en BD antigua: 71
Total de anotaciones: 99001

Top 10:


,BookID,BookTitle,num_anotaciones
0,file:///mnt/onboard/.kobo/dropbox/The_Happines...,The Happiness Hypothesis,21580
1,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,Las 5 trampas del amor,9844
2,file:///mnt/onboard/.kobo/dropbox/TeorÃ_a del ...,TeorÃ­a del apego y psicoterapia: en busca de ...,6042
3,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,Surrounded by Idiots,5247
4,file:///mnt/onboard/.kobo/dropbox/Attached_ Ar...,"Attached: Are you Anxious, Avoidant or Secure?...",4553
5,file:///mnt/onboard/.kobo/dropbox/Models -- Ma...,Models,4368
6,file:///mnt/onboard/.kobo/dropbox/El_sutil_art...,El sutil arte de que (casi todo) te importe un...,3840
7,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,Fluir (Flow),2864
8,file:///mnt/onboard/.kobo/dropbox/Como_hacer_q...,Como hacer que te pasen cosas buenas,2800
9,file:///mnt/onboard/.kobo/dropbox/Como_analiza...,Cómo analizar a las personas y el lenguaje cor...,2480


## 3. Obtener libros en BD nueva

In [4]:
# Obtener todos los libros en BD nueva
libros_nuevo = db_nuevo.get_query_df("""
    SELECT DISTINCT 
        c.BookID,
        c.BookTitle
    FROM content c
    WHERE c.BookTitle IS NOT NULL
""")

print(f'Total de libros en BD nueva: {len(libros_nuevo)}')
print('\nPrimeros 10:')
display(libros_nuevo.head(10))

Total de libros en BD nueva: 227

Primeros 10:


,BookID,BookTitle
0,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,12 reglas para vivir
1,file:///mnt/onboard/.kobo/dropbox/1984_George_...,1984
2,file:///mnt/onboard/.kobo/dropbox/30_dias_Camb...,"30 días. Cambia de hábitos, cambia de vida"
3,file:///mnt/onboard/.kobo/dropbox/4_6014754127...,
4,file:///mnt/onboard/.kobo/dropbox/A Random Wal...,A Random Walk Down Wall Street
5,file:///mnt/onboard/.kobo/dropbox/A_puerta_cer...,A puerta cerrada
6,file:///mnt/onboard/.kobo/dropbox/Acontece_que...,Acontece que no es poco
7,file:///mnt/onboard/.kobo/dropbox/Amanecer_Con...,Amanecer
8,file:///mnt/onboard/.kobo/dropbox/Amor_zero_In...,Amor zero
9,file:///mnt/onboard/.kobo/dropbox/Attached_ Ar...,"Attached: Are you Anxious, Avoidant or Secure?..."


## 4. Función de similitud y mapeo

In [5]:
def similitud(s1, s2):
    """Calcula similitud entre dos strings"""
    if pd.isna(s1) or pd.isna(s2):
        return 0
    s1 = str(s1).lower().strip()
    s2 = str(s2).lower().strip()
    return SequenceMatcher(None, s1, s2).ratio()

def encontrar_libro_en_nuevo(titulo_antiguo, libros_df, umbral=0.85):
    """Busca el libro más similar en BD nueva"""
    scores = libros_df['BookTitle'].apply(lambda x: similitud(titulo_antiguo, x))
    mejor_idx = scores.idxmax()
    mejor_score = scores[mejor_idx]
    
    if mejor_score >= umbral:
        return libros_df.loc[mejor_idx, 'BookID'], mejor_score
    return None, mejor_score

print('✓ Funciones de mapeo creadas')

✓ Funciones de mapeo creadas


## 5. Buscar equivalentes en BD nueva

In [6]:
# Mapear cada libro antiguo con anotaciones al libro nuevo equivalente
mapeos = []

for idx, row in libros_con_anotaciones_antiguo.iterrows():
    titulo_antiguo = row['BookTitle']
    book_id_antiguo = row['BookID']
    num_anot = row['num_anotaciones']
    
    book_id_nuevo, score = encontrar_libro_en_nuevo(titulo_antiguo, libros_nuevo, umbral=0.80)
    
    mapeos.append({
        'titulo_antiguo': titulo_antiguo,
        'bookid_antiguo': book_id_antiguo,
        'bookid_nuevo': book_id_nuevo,
        'score_similitud': score,
        'num_anotaciones': num_anot,
        'encontrado': book_id_nuevo is not None
    })

mapeos_df = pd.DataFrame(mapeos)

print(f'\n=== RESUMEN DE MAPEO ===')
print(f'Libros con anotaciones en BD antigua: {len(mapeos_df)}')
print(f'Encontrados en BD nueva (similitud >= 0.80): {mapeos_df["encontrado"].sum()}')
print(f'NO encontrados: {(~mapeos_df["encontrado"]).sum()}')
print(f'Total de anotaciones a migrar: {mapeos_df[mapeos_df["encontrado"]]["num_anotaciones"].sum()}')

print('\n=== LIBROS ENCONTRADOS ===')
display(mapeos_df[mapeos_df['encontrado']].sort_values('score_similitud', ascending=False).head(20))


=== RESUMEN DE MAPEO ===
Libros con anotaciones en BD antigua: 71
Encontrados en BD nueva (similitud >= 0.80): 70
NO encontrados: 1
Total de anotaciones a migrar: 98893

=== LIBROS ENCONTRADOS ===


,titulo_antiguo,bookid_antiguo,bookid_nuevo,score_similitud,num_anotaciones,encontrado
0,The Happiness Hypothesis,file:///mnt/onboard/.kobo/dropbox/The_Happines...,file:///mnt/onboard/.kobo/dropbox/The_Happines...,1.0,21580,True
44,La ciudad de los prodigios,file:///mnt/onboard/.kobo/dropbox/La_ciudad_de...,file:///mnt/onboard/.kobo/dropbox/La_ciudad_de...,1.0,184,True
51,How to Win at Chess,file:///mnt/onboard/.kobo/dropbox/How to Win a...,file:///mnt/onboard/.kobo/dropbox/How to Win a...,1.0,107,True
49,Cómo ser un estoico,file:///mnt/onboard/.kobo/dropbox/Como_ser_un_...,file:///mnt/onboard/.kobo/dropbox/Como_ser_un_...,1.0,125,True
48,Psicología oscura,file:///mnt/onboard/.kobo/dropbox/Psicologia_o...,file:///mnt/onboard/.kobo/dropbox/Psicologia_o...,1.0,143,True
47,Las madres,file:///mnt/onboard/.kobo/dropbox/Las_madres_C...,file:///mnt/onboard/.kobo/dropbox/Las_madres_C...,1.0,158,True
46,Cicatriz,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,1.0,159,True
45,Amanecer,file:///mnt/onboard/.kobo/dropbox/Amanecer_Con...,file:///mnt/onboard/.kobo/dropbox/Amanecer_Con...,1.0,176,True
43,Una historia de España,file:///mnt/onboard/.kobo/dropbox/Una_historia...,file:///mnt/onboard/.kobo/dropbox/Una_historia...,1.0,192,True
53,Todo arde,file:///mnt/onboard/.kobo/dropbox/Todo_arde_Ju...,file:///mnt/onboard/.kobo/dropbox/Todo_arde_Ju...,1.0,105,True


## 6. Libros NO encontrados (problema)

In [7]:
no_encontrados = mapeos_df[~mapeos_df['encontrado']]

if len(no_encontrados) > 0:
    print(f'⚠️  LIBROS CON ANOTACIONES NO ENCONTRADOS EN BD NUEVA: {len(no_encontrados)}')
    print(f'Anotaciones perdidas: {no_encontrados["num_anotaciones"].sum()}')
    print('\nDetalles:')
    display(no_encontrados[['titulo_antiguo', 'num_anotaciones', 'score_similitud']].head(20))
else:
    print('✅ TODOS los libros con anotaciones en BD antigua existen en BD nueva')

⚠️  LIBROS CON ANOTACIONES NO ENCONTRADOS EN BD NUEVA: 1
Anotaciones perdidas: 108

Detalles:


,titulo_antiguo,num_anotaciones,score_similitud
50,El juego del alma,108,0.628571


## 7. Obtener anotaciones de BD antigua

In [8]:
# Obtener TODAS las anotaciones de BD antigua
anotaciones_antiguas = db_antiguo.get_query_df("""
    SELECT *
    FROM Bookmark
    WHERE VolumeID IN (SELECT DISTINCT BookID FROM content WHERE BookID IS NOT NULL)
""")

print(f'Total de anotaciones en BD antigua: {len(anotaciones_antiguas)}')
print(f'\nColumnas en tabla Bookmark:')
print(anotaciones_antiguas.columns.tolist())
print(f'\nPrimera anotación:')
display(anotaciones_antiguas.iloc[0:1])

Total de anotaciones en BD antigua: 2652

Columnas en tabla Bookmark:
['BookmarkID', 'VolumeID', 'ContentID', 'StartContainerPath', 'StartContainerChildIndex', 'StartOffset', 'EndContainerPath', 'EndContainerChildIndex', 'EndOffset', 'Text', 'Annotation', 'ExtraAnnotationData', 'DateCreated', 'ChapterProgress', 'Hidden', 'Version', 'DateModified', 'Creator', 'UUID', 'UserID', 'SyncTime', 'Published', 'ContextString', 'Type']

Primera anotación:


,BookmarkID,VolumeID,ContentID,StartContainerPath,StartContainerChildIndex,StartOffset,EndContainerPath,EndContainerChildIndex,EndOffset,Text,Annotation,ExtraAnnotationData,DateCreated,ChapterProgress,Hidden,Version,DateModified,Creator,UUID,UserID,SyncTime,Published,ContextString,Type
0,2cf04d87-e270-4da9-9569-a98f5b7b1340,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,0,0,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,0,0,None,None,OEBPS/Text/cap007.xhtml#point(/1/4/90/1:85),2024-08-27T14:07:42.016,0.464373,false,None,2024-08-27T14:07:42Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear


## 8. Función para copiar anotaciones con remapeo de IDs

In [27]:
def copiar_anotaciones_con_remapeo(db_origen, db_destino, mapeos_df, dry_run=True, verbose=False):
    """
    Copia anotaciones de BD origen a BD destino, remapeando los IDs de libros.

    Devuelve además métricas reales de inserción (success/fail).
    """

    # Crear diccionario de mapeo
    mapeo_ids = dict(zip(
        mapeos_df[mapeos_df['encontrado']]['bookid_antiguo'],
        mapeos_df[mapeos_df['encontrado']]['bookid_nuevo']
    ))

    resumen = []
    total_anotaciones = 0
    success_count = 0
    fail_count = 0
    fail_examples = []

    # Obtener columnas y pk de la tabla Bookmark en DB destino
    try:
        pragma = db_destino.get_query_df("PRAGMA table_info('Bookmark')")
        pk_cols = pragma.loc[pragma['pk'] > 0, 'name'].tolist()
        all_cols = pragma['name'].tolist()
    except Exception:
        pk_cols = []
        all_cols = None

    # Columnas que usaremos para detectar duplicados (si están presentes)
    dup_candidates = ['VolumeID', 'ContentID', 'StartContainerPath', 'Text', 'DateCreated']

    for book_id_antiguo, book_id_nuevo in mapeo_ids.items():
        # Obtener anotaciones del libro antiguo
        query = "SELECT * FROM Bookmark WHERE VolumeID = ?"
        anotaciones = db_origen.get_query_df(query, params=(book_id_antiguo,))

        if len(anotaciones) > 0:
            # Cambiar VolumeID al nuevo
            anotaciones_remapeadas = anotaciones.copy()
            anotaciones_remapeadas['VolumeID'] = book_id_nuevo

            num_anot = len(anotaciones_remapeadas)
            total_anotaciones += num_anot

            if not dry_run:
                # Preparar columnas para insert: excluir PKs si existen
                if all_cols is None:
                    insert_cols = list(anotaciones_remapeadas.columns)
                else:
                    # Necesitamos incluir BookmarkID aunque sea PK en la tabla destino,
                    # porque en este esquema BookmarkID es NOT NULL y no autogenerable.
                    insert_cols = [
                        c for c in anotaciones_remapeadas.columns
                        if c in all_cols and (c not in pk_cols or c == 'BookmarkID')
                    ]

                for i, (_, anot_row) in enumerate(anotaciones_remapeadas.iterrows()):
                    try:
                        # Construir criterio de existencia con candidatos disponibles
                        where_clauses = []
                        params = []
                        for c in dup_candidates:
                            if c in anotaciones_remapeadas.columns:
                                val = anot_row.get(c, None)
                                if pd.notna(val) and val != '':
                                    where_clauses.append(f'\"{c}\" = ?')
                                    params.append(val)

                        exists = False
                        if where_clauses:
                            where_sql = ' AND '.join(where_clauses)
                            exists_q = f"SELECT COUNT(*) as total FROM Bookmark WHERE {where_sql}"
                            exists_res = db_destino.get_query_df(exists_q, params=tuple(params))
                            exists = int(exists_res['total'].values[0]) > 0

                        if exists:
                            # Ya hay una anotación equivalente; omitir
                            if verbose:
                                print('Omitiendo duplicado basado en campos clave')
                            continue

                        # Insertar: usar columnas sin PK (salvo BookmarkID)
                        cols_sql = ', '.join([f'\"{c}\"' for c in insert_cols])
                        placeholders = ', '.join(['?' for _ in insert_cols])
                        sql = f"INSERT INTO Bookmark ({cols_sql}) VALUES ({placeholders})"

                        # Convertir valores a nativos de Python para sqlite
                        values = []
                        for c in insert_cols:
                            v = anot_row.get(c, None)
                            if pd.isna(v):
                                values.append(None)
                            else:
                                # Convertir numpy types
                                try:
                                    values.append(v.item() if hasattr(v, 'item') else v)
                                except Exception:
                                    values.append(v)

                        db_destino.execute_non_query(sql, params=tuple(values))
                        success_count += 1

                    except Exception as e:
                        fail_count += 1
                        if len(fail_examples) < 10:
                            fail_examples.append(str(e))
                        if verbose:
                            print(f'    Error insertando anotación: {e}')

            resumen.append({
                'libro_antiguo_id': book_id_antiguo[:60] + '...' if len(str(book_id_antiguo)) > 60 else book_id_antiguo,
                'libro_nuevo_id': book_id_nuevo[:60] + '...' if len(str(book_id_nuevo)) > 60 else book_id_nuevo,
                'anotaciones': num_anot
            })

    resumen_df = pd.DataFrame(resumen)

    metrics = {
        'attempted': total_anotaciones,
        'inserted': success_count,
        'failed': fail_count,
        'fail_examples': fail_examples
    }

    return resumen_df, metrics

print('✓ Función copiar_anotaciones_con_remapeo actualizada (BookmarkID incluido cuando hace falta)')

✓ Función copiar_anotaciones_con_remapeo actualizada (BookmarkID incluido cuando hace falta)


## 9. Simular migración (DRY RUN)

In [17]:
print('=== SIMULACIÓN (DRY RUN) ===')
print('\nAnotaciones que se copiarían:')

resumen_simulado, total_sim = copiar_anotaciones_con_remapeo(
    db_antiguo, db_nuevo, mapeos_df, dry_run=True
)

display(resumen_simulado)
print(f'\n✓ Total de anotaciones a copiar: {total_sim}')

=== SIMULACIÓN (DRY RUN) ===

Anotaciones que se copiarían:


,libro_antiguo_id,libro_nuevo_id,anotaciones
0,file:///mnt/onboard/.kobo/dropbox/The_Happines...,file:///mnt/onboard/.kobo/dropbox/The_Happines...,260
1,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,92
2,file:///mnt/onboard/.kobo/dropbox/TeorÃ_a del ...,file:///mnt/onboard/.kobo/dropbox/TeorÃ_a del ...,57
3,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,159
4,file:///mnt/onboard/.kobo/dropbox/Attached_ Ar...,file:///mnt/onboard/.kobo/dropbox/Attached_ Ar...,157
5,file:///mnt/onboard/.kobo/dropbox/Models -- Ma...,file:///mnt/onboard/.kobo/dropbox/Models -- Ma...,156
6,file:///mnt/onboard/.kobo/dropbox/El_sutil_art...,file:///mnt/onboard/.kobo/dropbox/El_sutil_art...,80
7,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,179
8,file:///mnt/onboard/.kobo/dropbox/Como_hacer_q...,file:///mnt/onboard/.kobo/dropbox/Como_hacer_q...,35
9,file:///mnt/onboard/.kobo/dropbox/Como_analiza...,file:///mnt/onboard/.kobo/dropbox/Como_analiza...,155



✓ Total de anotaciones a copiar: 2650


## 10. APLICAR MIGRACIÓN (ACTUAL)

## 9.5 DEBUG: Verificar mapeos y duplicados antes de migrar

In [12]:
# Verificar si los libros mapeados tienen anotaciones DUPLICADAS
# Es decir, ¿hay libros que aparecen en AMBAS BDs con el MISMO ID?

print('=== DEBUG: Análisis de Duplicados ===')

# Obtener BookIDs que ya existen en BD nueva
bookids_nuevo_existentes = db_nuevo.get_query_df("""
    SELECT DISTINCT VolumeID FROM Bookmark
""")['VolumeID'].unique()

print(f'BookIDs con anotaciones en BD nueva ANTES de migrar: {len(bookids_nuevo_existentes)}')

# Obtener BookIDs mapeados (que se van a copiar)
mapeo_ids_dict = dict(zip(
    mapeos_df[mapeos_df['encontrado']]['bookid_antiguo'],
    mapeos_df[mapeos_df['encontrado']]['bookid_nuevo']
))

bookids_a_copiar = list(mapeo_ids_dict.values())
print(f'BookIDs que se van a copiar (nuevos): {len(bookids_a_copiar)}')

# Detectar OVERLAP: BookIDs que YA existen en BD nueva y TAMBIÉN se van a copiar
overlap = set(bookids_nuevo_existentes) & set(bookids_a_copiar)
print(f'BookIDs que YA EXISTEN en BD nueva y se van a "copiar": {len(overlap)}')

if len(overlap) > 0:
    print('\n⚠️  PROBLEMA DETECTADO: Hay anotaciones duplicadas que se intentarán insertar')
    print('Ejemplos de BookIDs duplicados:')
    for bid in list(overlap)[:5]:
        # Contar cuántas anotaciones hay de este libro en cada BD
        anot_nuevo = db_nuevo.get_query_df(f"SELECT COUNT(*) as total FROM Bookmark WHERE VolumeID = ?", params=(bid,))['total'].values[0]
        anot_antiguo = db_antiguo.get_query_df(f"SELECT COUNT(*) as total FROM Bookmark WHERE VolumeID = ?", params=(bid,))['total'].values[0]
        print(f'  - {bid[:60]}...')
        print(f'    En BD nueva: {anot_nuevo} anotaciones')
        print(f'    En BD antigua: {anot_antiguo} anotaciones')
else:
    print('\n✅ No hay duplicados: los libros a copiar son nuevos en BD nueva')

=== DEBUG: Análisis de Duplicados ===
BookIDs con anotaciones en BD nueva ANTES de migrar: 2
BookIDs que se van a copiar (nuevos): 70
BookIDs que YA EXISTEN en BD nueva y se van a "copiar": 0

✅ No hay duplicados: los libros a copiar son nuevos en BD nueva


In [29]:
# CUIDADO: Esta celda MODIFICA la BD nueva
# Antes de ejecutar, hacer backup:
# cp data/KoboReader_nuevo.sqlite data/KoboReader_nuevo.sqlite.bak

print('\n⚠️  ADVERTENCIA: Esta celda va a MODIFICAR data/KoboReader_nuevo.sqlite')
print('Se recomienda hacer backup primero:')
print('  cp data/KoboReader_nuevo.sqlite data/KoboReader_nuevo.sqlite.bak')
print('\n¿Deseas continuar? (descomentar y ejecutar la siguiente línea para aplicar cambios)')
print('\nAhora ejecutando DRY RUN...')
print('Si todo se ve correcto, descomenta las líneas de abajo para APLICAR:')
print()
print("""# DESCOMENTA ESTAS LÍNEAS PARA APLICAR:
# resumen_real, total_real = copiar_anotaciones_con_remapeo(
#     db_antiguo, db_nuevo, mapeos_df, dry_run=False
# )
# print(f'✓ Migración completada: {total_real} anotaciones copiadas')
""")
resumen_real, total_real = copiar_anotaciones_con_remapeo(
    db_antiguo, db_nuevo, mapeos_df, dry_run=False
)
print(f'✓ Migración completada: {total_real} anotaciones copiadas')



⚠️  ADVERTENCIA: Esta celda va a MODIFICAR data/KoboReader_nuevo.sqlite
Se recomienda hacer backup primero:
  cp data/KoboReader_nuevo.sqlite data/KoboReader_nuevo.sqlite.bak

¿Deseas continuar? (descomentar y ejecutar la siguiente línea para aplicar cambios)

Ahora ejecutando DRY RUN...
Si todo se ve correcto, descomenta las líneas de abajo para APLICAR:

# DESCOMENTA ESTAS LÍNEAS PARA APLICAR:
# resumen_real, total_real = copiar_anotaciones_con_remapeo(
#     db_antiguo, db_nuevo, mapeos_df, dry_run=False
# )
# print(f'✓ Migración completada: {total_real} anotaciones copiadas')

✓ Migración completada: {'attempted': 2650, 'inserted': 2390, 'failed': 0, 'fail_examples': []} anotaciones copiadas
✓ Migración completada: {'attempted': 2650, 'inserted': 2390, 'failed': 0, 'fail_examples': []} anotaciones copiadas


## 11. Verificación post-migración

In [31]:
# Verificar cuántas anotaciones hay ahora en BD nueva DESPUÉS de la migración
total_anot_nuevo_despues = db_nuevo.get_query_df("SELECT COUNT(*) as total FROM Bookmark")

# Conteo inicial antes de migrar (2 anotaciones)
initial_count = 2

print('=== VERIFICACIÓN POST-MIGRACIÓN ===')
print(f'Anotaciones INICIALES en BD nueva: {initial_count}')
print(f'Anotaciones INSERTADAS exitosamente: {2390}')
print(f'Total esperado en BD nueva: {initial_count + 2390}')
print(f'Total obtenido en BD nueva: {total_anot_nuevo_despues["total"].values[0]}')
print(f'Anotaciones en BD antigua: 2652')
print()

actual_count = total_anot_nuevo_despues["total"].values[0]
expected_count = initial_count + 2390

print('⚠️  DIAGNÓSTICO:')
if actual_count == expected_count:
    print(f'✅ La migración se hizo correctamente ({2390} anotaciones copiadas)')
elif actual_count == initial_count:
    print(f'❌ ERROR: Las anotaciones NO se insertaron (aún hay {actual_count})')
else:
    print(f'⚠️  Discrepancia: se esperaba {expected_count}, se obtuvieron {actual_count}')
    print(f'   Diferencia: {actual_count - expected_count}')

=== VERIFICACIÓN POST-MIGRACIÓN ===
Anotaciones INICIALES en BD nueva: 2
Anotaciones INSERTADAS exitosamente: 2390
Total esperado en BD nueva: 2392
Total obtenido en BD nueva: 2652
Anotaciones en BD antigua: 2652

⚠️  DIAGNÓSTICO:
⚠️  Discrepancia: se esperaba 2392, se obtuvieron 2652
   Diferencia: 260


In [25]:
# DIAGNÓSTICO ADICIONAL: mostrar detalles después de la migración
print('DB NUEVO PATH:', db_nuevo.db_path if hasattr(db_nuevo, 'db_path') else getattr(db_nuevo, 'db_path', 'desconocido'))
print('DB ANTIGUO PATH:', db_antiguo.db_path if hasattr(db_antiguo, 'db_path') else getattr(db_antiguo, 'db_path', 'desconocido'))

print('\n-- Primeros 20 Bookmark en BD nueva --')
df_nuevo_bookmarks = db_nuevo.get_query_df("SELECT * FROM Bookmark LIMIT 20")
display(df_nuevo_bookmarks)
print('\nTotal en BD nueva (consulta directa):', db_nuevo.get_query_df("SELECT COUNT(*) as total FROM Bookmark")['total'].values[0])

print('\n-- Primeros 20 Bookmark en BD antigua --')
df_ant_bookmarks = db_antiguo.get_query_df("SELECT * FROM Bookmark LIMIT 20")
display(df_ant_bookmarks)
print('\nTotal en BD antigua (consulta directa):', db_antiguo.get_query_df("SELECT COUNT(*) as total FROM Bookmark")['total'].values[0])

# Mostrar resumen_real y total_real si existen
try:
    print('\nresumen_real (head):')
    display(resumen_real.head(20))
    print('total_real:', total_real)
except NameError:
    print('\nNo existe variable resumen_real/total_real en el kernel')

# Mostrar mensajes de error parciales (si hubo prints durante la inserción, los verás en el output anterior)

DB NUEVO PATH: data/KoboReader_nuevo.sqlite
DB ANTIGUO PATH: data/KoboReader.sqlite

-- Primeros 20 Bookmark en BD nueva --


,BookmarkID,VolumeID,ContentID,StartContainerPath,StartContainerChildIndex,StartOffset,EndContainerPath,EndContainerChildIndex,EndOffset,Text,Annotation,ExtraAnnotationData,DateCreated,ChapterProgress,Hidden,Version,DateModified,Creator,UUID,UserID,SyncTime,Published,ContextString,Type
0,b07859c8-48e2-48c5-a302-8fc5bd967c2e,file:///mnt/onboard/.kobo/dropbox/El_lobo_de_W...,file:///mnt/onboard/.kobo/dropbox/El_lobo_de_W...,OEBPS/Text/002.xhtml#point(/1/4/212/1:353),-99,0,OEBPS/Text/002.xhtml#point(/1/4/212/1:433),-99,0,"con otras personas, me tomaba de la mano, o me...",,None,2025-11-19T16:04:01.465,0.079245,false,None,None,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight
1,1915c3a5-adb1-4c27-8bd7-e5975a8ef4d6,file:///mnt/onboard/.kobo/dropbox/Once_anillos...,file:///mnt/onboard/.kobo/dropbox/Once_anillos...,OEBPS/Text/sinopsis.xhtml#point(/1/4/2/6/1:467),-99,0,OEBPS/Text/sinopsis.xhtml#point(/1/4/2/6/1:507),-99,0,creer en el trabajo en equipo por encima,,None,2025-11-19T17:23:24.123,0.006472,false,None,None,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,highlight



Total en BD nueva (consulta directa): 2

-- Primeros 20 Bookmark en BD antigua --


,BookmarkID,VolumeID,ContentID,StartContainerPath,StartContainerChildIndex,StartOffset,EndContainerPath,EndContainerChildIndex,EndOffset,Text,Annotation,ExtraAnnotationData,DateCreated,ChapterProgress,Hidden,Version,DateModified,Creator,UUID,UserID,SyncTime,Published,ContextString,Type
0,a5090ecd-389f-4167-b499-d4bb11d75c21,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,0,0,file:///mnt/onboard/.kobo/dropbox/Legado_en_lo...,0,0,None,None,OEBPS/Text/007.xhtml#point(/1/4/278/1:454),2022-03-21T21:03:49.079,0.206573,false,None,2022-03-21T21:03:49Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
1,d70ed735-e49b-4097-a5ea-29d87ba55d4c,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,0,0,file:///mnt/onboard/.kobo/dropbox/Loba_Negra_J...,0,0,None,None,OEBPS/Text/Intro.xhtml#point(/1/4/4/22/1:257),2022-08-14T16:02:13.741,0.288770,false,None,2022-08-14T16:02:13Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
2,ab6d7a39-6ae0-4073-84d3-536c1a32a1be,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,0,0,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,0,0,None,None,OEBPS/Text/part0041.xhtml#point(/1/4/46/1:0),2022-08-16T16:11:46.015,0.450575,false,None,2022-08-16T16:11:46Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
3,e44bc853-5125-4441-bb62-1855ba97b424,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,OEBPS/Text/Sec0208.xhtml#point(/1/4/28/1:186),0,0,OEBPS/Text/Sec0208.xhtml#point(/1/4/28/1:186),0,0,None,None,       S t a r t K e y             S c r e ...,2023-01-13T19:08:37.864,0.453988,false,None,2023-01-13T19:08:37Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,markup
4,1734a906-4785-4ef4-9880-a8d4350e1b2e,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,0,0,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,0,0,None,None,OEBPS/Text/Sec0202.xhtml#point(/1/4/30/1:0),2023-01-10T23:04:12.015,0.226994,false,None,2023-01-10T23:04:12Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
5,1624b836-4616-4205-98ba-d8d4b2b59751,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,OEBPS/Text/Sec0208.xhtml#point(/1/4/20/1:46),0,0,OEBPS/Text/Sec0208.xhtml#point(/1/4/20/1:46),0,0,None,None,       S t a r t K e y             S c r e ...,2023-01-13T19:08:30.064,0.453988,false,None,2023-01-13T19:08:30Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,markup
6,0ea2260f-a66b-4ec1-9645-c9d0b9328e96,file:///mnt/onboard/.kobo/dropbox/Rey_Blanco_J...,file:///mnt/onboard/.kobo/dropbox/Rey_Blanco_J...,file:///mnt/onboard/.kobo/dropbox/Rey_Blanco_J...,0,0,file:///mnt/onboard/.kobo/dropbox/Rey_Blanco_J...,0,0,None,None,OEBPS/Text/Cat22.xhtml#point(/1/4/28/1:0),2023-01-06T01:20:32.230,0.941667,false,None,2023-01-06T01:20:32Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
7,27bf7f33-04a3-4ac5-a0d5-c084d9695ee1,file:///mnt/onboard/.kobo/dropbox/Sin_noticias...,file:///mnt/onboard/.kobo/dropbox/Sin_noticias...,file:///mnt/onboard/.kobo/dropbox/Sin_noticias...,0,0,file:///mnt/onboard/.kobo/dropbox/Sin_noticias...,0,0,None,None,OEBPS/Text/20.xhtml#point(/1/4/2/1:0),2023-01-26T01:46:43.515,0.612245,false,None,2023-01-26T01:46:43Z,None,None,83b63413-c6de-45c8-a874-7cadf24716d7,None,false,None,dogear
8,9e42a541-20a4-491a-b8c8-2055a92f078b,file:///mnt/onboard/.kobo/dropbox/La_verdad_so...,file:///mnt/onboard/.kobo/dropbox/La_verdad_so...,OEBPS/Text/S03.xhtml#point(/1/4/50/1:24),0,0,OEBPS/Text/S03.xhtml#point(/1/4/50/1:27),0,0,None,Non


Total en BD antigua (consulta directa): 2652

resumen_real (head):


,libro_antiguo_id,libro_nuevo_id,anotaciones
0,file:///mnt/onboard/.kobo/dropbox/The_Happines...,file:///mnt/onboard/.kobo/dropbox/The_Happines...,260
1,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,92
2,file:///mnt/onboard/.kobo/dropbox/TeorÃ_a del ...,file:///mnt/onboard/.kobo/dropbox/TeorÃ_a del ...,57
3,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,159
4,file:///mnt/onboard/.kobo/dropbox/Attached_ Ar...,file:///mnt/onboard/.kobo/dropbox/Attached_ Ar...,157
5,file:///mnt/onboard/.kobo/dropbox/Models -- Ma...,file:///mnt/onboard/.kobo/dropbox/Models -- Ma...,156
6,file:///mnt/onboard/.kobo/dropbox/El_sutil_art...,file:///mnt/onboard/.kobo/dropbox/El_sutil_art...,80
7,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,179
8,file:///mnt/onboard/.kobo/dropbox/Como_hacer_q...,file:///mnt/onboard/.kobo/dropbox/Como_hacer_q...,35
9,file:///mnt/onboard/.kobo/dropbox/Como_analiza...,file:///mnt/onboard/.kobo/dropbox/Como_analiza...,155


total_real: 2650


## 12. Cerrar conexiones

In [15]:
db_nuevo.close()
db_antiguo.close()

print('✓ Conexiones cerradas')

✓ Conexiones cerradas


In [28]:
# PRUEBA: insertar anotaciones para UN libro (mapeo simple)
print('== PRUEBA: inserción para 1 libro ==')
map_found = mapeos_df[mapeos_df['encontrado']]
if len(map_found) == 0:
    print('No hay mapeos encontrados para probar')
else:
    mapping = map_found.iloc[0]
    mapeo_small = pd.DataFrame([mapping])
    print('Libro a probar (antiguo):', mapping['titulo_antiguo'])
    print('BookID nuevo objetivo:', mapping['bookid_nuevo'])

    resumen_test, metrics_test = copiar_anotaciones_con_remapeo(db_antiguo, db_nuevo, mapeo_small, dry_run=False, verbose=True)
    print('\nMetrics test:')
    print(metrics_test)

    bid_nuevo = mapping['bookid_nuevo']
    count_dest = db_nuevo.get_query_df("SELECT COUNT(*) as total FROM Bookmark WHERE VolumeID = ?", params=(bid_nuevo,))['total'].values[0]
    print('Conteo en BD nueva para bookid_nuevo:', count_dest)


== PRUEBA: inserción para 1 libro ==
Libro a probar (antiguo): The Happiness Hypothesis
BookID nuevo objetivo: file:///mnt/onboard/.kobo/dropbox/The_Happiness_Hypothesis.epub

Metrics test:
{'attempted': 260, 'inserted': 260, 'failed': 0, 'fail_examples': []}
Conteo en BD nueva para bookid_nuevo: 260


In [26]:
# Mostrar métricas de la prueba
print('metrics_test:')
try:
    print(metrics_test)
except NameError:
    print('metrics_test no existe')

print('\nresumen_test (head):')
try:
    display(resumen_test.head())
except NameError:
    print('resumen_test no existe')

print('\ncount_dest (para bookid probado):')
try:
    print(int(count_dest))
except NameError:
    print('count_dest no existe')


metrics_test:
{'attempted': 260, 'inserted': 0, 'failed': 260, 'fail_examples': ['NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID', 'NOT NULL constraint failed: Bookmark.BookmarkID']}

resumen_test (head):


,libro_antiguo_id,libro_nuevo_id,anotaciones
0,file:///mnt/onboard/.kobo/dropbox/The_Happines...,file:///mnt/onboard/.kobo/dropbox/The_Happines...,260



count_dest (para bookid probado):
0


In [32]:
# Análisis: ¿de dónde vienen las 260 anotaciones adicionales?
print('=== ANÁLISIS DE DISCREPANCIA ===')
print('Esperado: 2 (iniciales) + 2390 (migradas con la función) = 2392')
print('Obtenido: 2652')
print('Diferencia: 260 anotaciones adicionales')
print()
print('El primer libro probado (The Happiness Hypothesis) tenía 260 anotaciones.')
print('→ Probablemente se insertó 2 veces (durante la prueba y durante la migración real)')
print('→ OR bien el conteo de 2390 "insertadas" no es del total de intentos sino de éxitos reales.')
print()

# Re-contar con más detalle: agrupar por VolumeID
print('Anotaciones por libro en BD nueva (post-migración):')
book_counts = db_nuevo.get_query_df("""
    SELECT VolumeID, COUNT(*) as total
    FROM Bookmark
    GROUP BY VolumeID
    ORDER BY total DESC
    LIMIT 15
""")
display(book_counts)
print()

# Verificar si alguno coincide con el conteo esperado del mapeo
print('Comparar contra resumen esperado de migración:')
print('Primer libro en mapeo (The Happiness Hypothesis):')
if len(resumen_real) > 0:
    print(resumen_real.iloc[0])
    print(f'→ Esperado {resumen_real.iloc[0]["anotaciones"]} anotaciones para este libro')


=== ANÁLISIS DE DISCREPANCIA ===
Esperado: 2 (iniciales) + 2390 (migradas con la función) = 2392
Obtenido: 2652
Diferencia: 260 anotaciones adicionales

El primer libro probado (The Happiness Hypothesis) tenía 260 anotaciones.
→ Probablemente se insertó 2 veces (durante la prueba y durante la migración real)
→ OR bien el conteo de 2390 "insertadas" no es del total de intentos sino de éxitos reales.

Anotaciones por libro en BD nueva (post-migración):


,VolumeID,total
0,file:///mnt/onboard/.kobo/dropbox/The_Happines...,260
1,file:///mnt/onboard/.kobo/dropbox/Fluir_Flow_M...,179
2,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,159
3,file:///mnt/onboard/.kobo/dropbox/Attached_ Ar...,157
4,file:///mnt/onboard/.kobo/dropbox/Models -- Ma...,156
5,file:///mnt/onboard/.kobo/dropbox/Como_analiza...,155
6,file:///mnt/onboard/.kobo/dropbox/El_arte_de_a...,144
7,file:///mnt/onboard/.kobo/dropbox/Las 5 trampa...,92
8,file:///mnt/onboard/.kobo/dropbox/El_sutil_art...,80
9,file:///mnt/onboard/.kobo/dropbox/Mindset _ Th...,80



Comparar contra resumen esperado de migración:
Primer libro en mapeo (The Happiness Hypothesis):
libro_antiguo_id    file:///mnt/onboard/.kobo/dropbox/The_Happines...
libro_nuevo_id      file:///mnt/onboard/.kobo/dropbox/The_Happines...
anotaciones                                                       260
Name: 0, dtype: object
→ Esperado 260 anotaciones para este libro


In [33]:
# Sumar el resumen esperado y comparar contra obtenido
print('=== COMPARACIÓN FINAL ===')
print()
print('Totales por libro según resumen_real (migración):')
suma_esperada = resumen_real['anotaciones'].sum()
print(f'  Suma: {suma_esperada}')
print()

# Contar totales en BD nueva agrupado
suma_obtenida = db_nuevo.get_query_df("""
    SELECT SUM(total) as suma FROM (
        SELECT COUNT(*) as total
        FROM Bookmark
        GROUP BY VolumeID
    ) t
""")['suma'].values[0]
print(f'Total anotaciones en BD nueva (conteo directo): {actual_count}')
print()

print('RESUMEN:')
print(f'  - 2 anotaciones iniciales')
print(f'  - {suma_esperada} anotaciones según resumen de migración')
print(f'  - Total ESPERADO: {2 + suma_esperada}')
print(f'  - Total OBTENIDO: {actual_count}')
print()

# Las 260 extra son porque ya se insertaron en la prueba anterior
extra_count = actual_count - (2 + suma_esperada)
print(f'Anotaciones EXTRA (probablemente de la prueba anterior): {extra_count}')

if extra_count == 260:
    print('✓ Coincide con las 260 anotaciones de la PRUEBA anterior')
    print('✓ La migración real FUNCIONÓ correctamente')
    print(f'✓ Se han migrado {suma_esperada} anotaciones de {len(resumen_real)} libros')


=== COMPARACIÓN FINAL ===

Totales por libro según resumen_real (migración):
  Suma: 2650

Total anotaciones en BD nueva (conteo directo): 2652

RESUMEN:
  - 2 anotaciones iniciales
  - 2650 anotaciones según resumen de migración
  - Total ESPERADO: 2652
  - Total OBTENIDO: 2652

Anotaciones EXTRA (probablemente de la prueba anterior): 0
